# Ch 20: Base and Standard Library

`import Module` imports the module, and `Module.fn(x)` calls the function `f`

### Measuring Performance

We have seen that some algorithms perform better than others. `fibonnaci` with memos is a lot faster than `fib` without memoization. The @time macro allows to quantify the difference.

@time prints the time the function took to execute, the number of allocations and the allocated memory before returning the result. The memoized version is effectively a lot faster but needs more memory.

TIP:  A function in Julia is compiled the first time it is executed. So to compare two algorithms, they have to be implemented as functions to get compiled and the first time they are called has to be excluded from the performance measure, otherwise the compilation time is measured as well. The package BenchmarkTools (https://github.com/JuliaCI/BenchmarkTools.jl) provides the macro @btime that does benchmarking the right way. So just use it!

In [11]:
known = Dict(0=>0, 1=>1)

function fibonacci(n)
    if n ∈ keys(known)
        return known[n]
    end
    res = fibonacci(n-1) + fibonacci(n-2)
    known[n] = res
    res
end



function fib(n)
    if n == 0
        return 0
    elseif n == 1
        return 1
    else
        return fib(n-1) + fib(n-2)
    end
end

fib (generic function with 1 method)

In [1]:
using Pkg
Pkg.add("BenchmarkTools")
using BenchmarkTools

  Updating registry at `C:\Users\ST\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `C:\Users\ST\.julia\environments\v1.2\Project.toml`
 [no changes]
  Updating `C:\Users\ST\.julia\environments\v1.2\Manifest.toml`
 [no changes]


In [12]:
n = 25
@btime fibonacci(n)

  113.597 ns (2 allocations: 32 bytes)


75025

In [13]:
@btime fib(n)

  607.399 μs (1 allocation: 16 bytes)


75025

### Collections and Data Structures

In Dictionary Subtraction we used dictionaries to find the words that appear in a document but not in a word array. The function we wrote takes `d1`, which contains the words from the document as keys, and `d2`, which contains the array of words. It returns a dictionary that contains the keys from `d1` that are not in `d2`.

In [4]:
function subtract(d1, d2)
    res = Dict()
    for key in keys(d1)
        if key ∉ keys(d2)
            res[key] = nothing
        end
    end
    res
end

subtract (generic function with 1 method)

In all of these dictionaries, the values are `nothing` because we never use them. As a result, we waste some storage space. Julia provides another built-in type, called a set, that behaves like a collection of dictionary keys with no values. Adding elements to a set is fast; so is checking membership. And sets provide functions and operators to compute common set operations. 

For example, set subtraction is available as a function called `setdiff`. So we can rewrite subtract like this:

In [5]:
function subtract(d1, d2)
    setdiff(d1, d2)
end

subtract (generic function with 1 method)

The result is a set instead of a dictionary.

Some of the exercises in this book can be done concisely and efficiently with sets. For example, here is a solution to `hasduplicates`, from Exercise 10-7, that uses a dictionary:

In [6]:
function hasduplicates(t)
    d = Dict()
    for x in t
        if x ∈ d
            return true
        end
        d[x] = nothing
    end
    false
end

hasduplicates (generic function with 1 method)

When an element appears for the first time, it is added to the dictionary. If the same element appears again, the function returns `true`.

Using sets, we can write the same function like this:

In [7]:
function hasduplicates(t)
    length(Set(t)) < length(t)
end

hasduplicates (generic function with 1 method)

An element can only appear in a set once, so if an element in `t` appears more than once, the set will be smaller than `t`. If
there are no duplicates, the set will be the same size as `t`.

We can also use sets to do some of the exercises in Case Study: Word Play. For example, here’s a version of usesonly with
a loop:


In [8]:
function usesonly(word, available)
    for letter in word
        if letter ∉ available
            return false
        end
    end
    true
end

usesonly (generic function with 1 method)

`usesonly` checks whether all letters in word are in `available`. We can rewrite it like this:

In [9]:
function usesonly(word, available)
    Set(word) ⊆ Set(available)
end

usesonly (generic function with 1 method)

### Exercise 20-1

Rewrite avoids using sets.

### Mathematics

Complex numbers are also supported in Julia. The global constant `im` is bound to the complex number $i$, representing the principal square root of $-1$.

We can now verify Euler’s identity,

In [10]:
ℯ^(im*π)+1

0.0 + 1.2246467991473532e-16im

The symbol ℯ ( \euler TAB ) is the base of natural logarithms

Let’s illustrate the complex nature of trigonometric functions:

$$\cos(x) = \frac{e^{ix} + e^{-ix}}{2}$$

In [14]:
x = 0:0.1:2π
cos.(x) == 0.5*(ℯ.^(im*x)+ℯ.^(-im*x))

true

Here, another example of the dot operator is shown. Julia also allows numeric literals to be juxtaposed with identifiers as
coefficients as in 2π.

### Strings

In Strings and Case Study: Word Play, we did some elementary searches in string objects. Julia can handle however Perl compatible regular expressions (regexes), which eases the task of finding complex patterns in string objets. The usesonly function can be implemented as a regex:

In [15]:
function usesonly(word, available)
    r = Regex("[^$(available)]")
    !occursin(r, word)
end

usesonly (generic function with 1 method)

The regex looks for a character that is not in the available string and occursin returns true if the pattern is found in
`word`.

In [16]:
usesonly("banana", "abn")

true

In [17]:
usesonly("bananas", "abn")

false

Regexes can also be constructed as non-standard string literals prefixed with `r`:

In [19]:
match(r"[^abn]", "banana")

In [20]:
m = match(r"[^abn]", "bananas")

RegexMatch("s")

String interpolation is not allowed in this case. The `match` function returns nothing if the pattern (a command) is not
found and return a regexmatch object otherwise.

We can extract the following info from a regexmatch object:

* the entire substring matched: m.match
* the captured substrings as an array of strings: m.captures
* the offset at which the whole match begins: m.offset
* the offsets of the captured substrings as an array: m.offsets

In [21]:
m.match

"s"

In [22]:
m.offset

7

Regexes are extremely powerful and the PERL manpage http://perldoc.perl.org/perlre.html provides all the details to
construct the most exotic searches.

### Arrays

In Arrays we used an array object as a one-dimensional container with an index to address its elements. In Julia however, arrays are multi-dimensional collections. 

Let’s create a 2-by-3 zero matrix:


In [23]:
z = zeros(Float64, 2, 3)

2×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0

In [24]:
typeof(z)

Array{Float64,2}

The type of this matrix is an array holding floating points and having 2 dimensions.

The size function returns a tuple with as elements the number of elements in each dimension:

In [25]:
size(z)

(2, 3)

The function `ones` constructs a matrix with unit value elements:

In [26]:
s = ones(String, 1, 3)

1×3 Array{String,2}:
 ""  ""  ""

The string unit element is an empty string.

**Warning**: `s` is not a one-dimensional array:

In [27]:
s == ["", "", ""]

false

`s` is a row matrix and ["", "", ""] is a column matrix.

A matrix can be entered directly using a space to separate elements in a row and a semicolon ; to separate rows:

In [29]:
a = [1 2 3; 4 5 6]

2×3 Array{Int64,2}:
 1  2  3
 4  5  6

You can use square brackets to address individual elements:

In [30]:
z[1,2] = 1

1

In [31]:
z[2,3] = 1

1

In [32]:
z

2×3 Array{Float64,2}:
 0.0  1.0  0.0
 0.0  0.0  1.0

Slices can be used for each dimension to select a subgroup of elements:

In [33]:
u = z[:,2:end]

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

The `.` operator broadcasts to all dimensions:

In [35]:
ℯ.^(im*u)

2×2 Array{Complex{Float64},2}:
 0.540302+0.841471im       1.0+0.0im     
      1.0+0.0im       0.540302+0.841471im

### Interfaces

Julia specifies some informal interfaces to define behaviors, i.e. methods with a specific goal. When you extend such a method for a type, objects of that type can be used to build upon these behaviors.

"If it looks like a duck, swims like a duck, and quacks like a duck, then it probably is a duck."

In One More Example we implemented the fib function returning the n-th element of the Fibonnaci sequence. Looping over the values of a collection, called iteration, is such an interface.

Let’s make an iterator that returns lazily the Fibonacci sequence:

In [36]:
struct Fibonacci{T<:Real} end

Fibonacci(d::DataType) = d<:Real ? Fibonacci{d}() : error("No Real type!")

Base.iterate(::Fibonacci{T}) where {T<:Real} = (zero(T), (one(T), one(T)))
Base.iterate(::Fibonacci{T}, state::Tuple{T, T}) where {T<:Real} = (state[1], (state[2], state[1] + state[2]))

We implemented a parametric type with no fields Fibonacci , an outer constructor and two methods iterate . The first is called to initialize the iterator and returns a tuple consisting of the first value, 0, and a state. The state in this case is a tuple containing the second and the third value, 1 and 1.

The second is called to get the next value of the Fibonacci sequence and returns a tuple having as first element the next value and as second element the state which is a tuple with the two following values. 

We can use Fibonacci now in a for loop:

In [37]:
for e in Fibonacci(Int64)
    e > 100 && break
    print(e, " ")
end

0 1 1 2 3 5 8 13 21 34 55 89 

It looks like magic has happened but the explanation is simple. A for loop in Julia

```
for i in iter
    # body
end
```

is translated into:

```
next = iterate(iter)
while next !== nothing
    (i, state) = next
    # body
    next = iterate(iter, state)
end
```

This is a great example how a well defined interface allows an implementation to use all the functions that are aware of the interface.


### Interactive Utilities

We have already met the InteractiveUtils module in Debugging. The @which macro is only the tip of the iceberg. Julia code is transformed by the LLVM library to machinecode in multiple steps. We can directly visualize the output of each stage.

Let’s give a simple example:

In [39]:
function squaresum(a::Float64, b::Float64)
    a^2 + b^2
end

squaresum (generic function with 1 method)

The first step is to look at the lowered code:

In [40]:
using InteractiveUtils
@code_lowered squaresum(3.0, 4.0)

CodeInfo(
1 ─ %1 = Core.apply_type(Base.Val, 2)
│   %2 = (%1)()
│   %3 = Base.literal_pow(Main.:^, a, %2)
│   %4 = Core.apply_type(Base.Val, 2)
│   %5 = (%4)()
│   %6 = Base.literal_pow(Main.:^, b, %5)
│   %7 = %3 + %6
└──      return %7
)

The @code_lowered macro returns an array of an intermediate representation of the code that is used by the compiler to generate optimised code.

The next step adds type information:

In [41]:
@code_typed squaresum(3.0, 4.0)

CodeInfo(
1 ─ %1 = Base.mul_float(a, a)::Float64
│   %2 = Base.mul_float(b, b)::Float64
│   %3 = Base.add_float(%1, %2)::Float64
└──      return %3
) => Float64

We see that the type of the intermediate results and the return value is correctly inferred.

This representation of the code is transformed into LLVM code:

In [42]:
@code_llvm squaresum(3.0, 4.0)


;  @ In[39]:2 within `squaresum'
; Function Attrs: uwtable
define double @julia_squaresum_17565(double, double) #0 {
top:
; ┌ @ intfuncs.jl:244 within `literal_pow'
; │┌ @ float.jl:399 within `*'
    %2 = fmul double %0, %0
    %3 = fmul double %1, %1
; └└
; ┌ @ float.jl:395 within `+'
   %4 = fadd double %2, %3
; └
  ret double %4
}


And finally the machine code is generated:

In [43]:
@code_native squaresum(3.0, 4.0)

	.text
; ┌ @ In[39]:2 within `squaresum'
	pushq	%rbp
	movq	%rsp, %rbp
; │┌ @ intfuncs.jl:244 within `literal_pow'
; ││┌ @ float.jl:399 within `*'
	vmulsd	%xmm0, %xmm0, %xmm0
	vmulsd	%xmm1, %xmm1, %xmm1
; │└└
; │┌ @ float.jl:395 within `+'
	vaddsd	%xmm1, %xmm0, %xmm0
; │└
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)
; └


### Debugging

The Logging macros provide an alternative to scaffolding with `print` statements:

In [44]:
@warn "Abandon printf debugging, all ye who enter here!"

┌ Warning: Abandon printf debugging, all ye who enter here!
└ @ Main In[44]:1


The debug statements don’t have to be removed from the source. For example, in contrast to the @warn above

In [45]:
@debug "The sum of some values $(sum(rand(100)))"

will produce no output by default. In this case sum(rand(100)) will never be evaluated unless debug logging is enabled.

The level of logging can be selected by an environment variable `JULIA_DEBUG`.  On the terminal

```
JULIA_DEBUG=all julia -e '@debug "The sum of some values $(sum(rand(100)))"'

```

Here, we have used all to get all debug information, but you can also choose to generate only output for a specific file or
module.